<a href="https://colab.research.google.com/github/Mahedi-Hasan-Anik/Thesis_materials/blob/main/Talha_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from glob import glob
import os
!pip install mne
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
all_files_path=glob('/content/drive/MyDrive/dataverse/*.edf')
print(len(all_files_path))

28


In [9]:
healthy_file_path=[i for i in all_files_path if  'h' in i.split('/content/drive/MyDrive/dataverse/')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('/content/drive/MyDrive/dataverse/')[1]]

In [10]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
    epochs=epochs.get_data()
    return epochs

In [ ]:
data=read_data(healthy_file_path[0])

In [ ]:
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [13]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [14]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

28 28


In [15]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

In [16]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(1142, 6250, 19) (1142,) (1142,)


In [ ]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(6250,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11

    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

In [18]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [19]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])


8/8 [==============================] - 5s 580ms/step - loss: 0.6768 - accuracy: 0.5267 - val_loss: 0.6484 - val_accuracy: 0.6456
Epoch 7/50
8/8 [==============================] - 8s 977ms/step - loss: 0.6740 - accuracy: 0.5267 - val_loss: 0.6443 - val_accuracy: 0.6456
Epoch 8/50
8/8 [==============================] - 9s 1s/step - loss: 0.6714 - accuracy: 0.5491 - val_loss: 0.6445 - val_accuracy: 0.6602
Epoch 9/50
8/8 [==============================] - 5s 551ms/step - loss: 0.6628 - accuracy: 0.5652 - val_loss: 0.6471 - val_accuracy: 0.7718
Epoch 10/50
8/8 [==============================] - 5s 692ms/step - loss: 0.6527 - accuracy: 0.5470 - val_loss: 0.6410 - val_accuracy: 0.7573
Epoch 11/50
8/8 [==============================] - 6s 733ms/step - loss: 0.6493 - accuracy: 0.5524 - val_loss: 0.6436 - val_accuracy: 0.6650
Epoch 12/50
8/8 [==============================] - 4s 490ms/step - loss: 0.6452 - accuracy: 0.5524 - val_loss: 0.6559 - val_accuracy: 0.5534
Epoch 13/50
8/8 [==============

In [20]:
np.mean(accuracy)

0.6519461870193481